# Q8 Prime Bias Experiment - Demo Notebook

Welcome to the Q8 Prime Bias Experiment! This notebook demonstrates how to use the complete numerical experiment environment for studying Chebyshev bias in Quaternion Galois extensions.

## 📚 Background

This implementation is based on:
**Aoki, M. and Koyama, S. (2022). Chebyshev's Bias against Splitting and Principal Primes in Global Fields. arXiv:2203.12266**

We study the bias in the distribution of Frobenius elements in Q8 (quaternion group) Galois extensions, implementing Example 2.1 and all 13 cases from Omar S.'s polynomial collection.

## 🎯 What you'll learn:
1. How to run Frobenius element computations
2. How to analyze bias using 5 different graph representations
3. How to interpret the results and compare with theory
4. How to scale up to 10^9 primes using parallel processing


## 🔧 Setup and Imports

In [ ]:
# Standard library imports
import sys
import os
import time
from pathlib import Path

# Add src directory to path
sys.path.append('./src')
sys.path.append('../src')  # In case notebook is in notebooks/ subdirectory

# Scientific computing
import numpy as np
import matplotlib.pyplot as plt

# Our Q8 experiment modules
from omar_polynomials import get_case, get_all_cases, print_all_cases_summary
from fast_frobenius_calculator import FastFrobeniusCalculator, ParallelFrobeniusComputation
from bias_analyzer import BiasAnalyzer
from experiment_runner import QuaternionBiasExperiment
from utils import format_number, format_duration, Timer

# Configure matplotlib for inline display
%matplotlib inline
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 12

print("✅ All modules imported successfully!")
print(f"Current working directory: {os.getcwd()}")

## 📊 Explore the 13 Polynomial Cases

Let's start by examining the 13 different Q8 Galois extension cases from Omar S.'s collection.

In [ ]:
# Display all 13 cases
print("Available Q8 Galois Extension Cases:")
print("====================================")
print_all_cases_summary()

# Get detailed information about Case 1 (Aoki-Koyama Example 2.1)
print("\n🔍 Detailed view of Case 1 (Aoki-Koyama Example 2.1):")
case1 = get_case(1)
print(f"Polynomial: {case1.polynomial}")
print(f"m_ρ₀ value: {case1.m_rho0}")
print(f"Ramified primes: {case1.ramified_primes}")
print(f"Bias coefficients: {case1.get_bias_coefficients()}")
print(f"Conjugacy class sizes: {case1.get_conjugacy_sizes()}")

## 🧪 Quick Test: Small Scale Computation

Let's start with a small-scale test to understand how the system works.

In [ ]:
# Initialize the experiment system
experiment = QuaternionBiasExperiment(verbose=True)

# Display system information
experiment.print_experiment_info()

In [ ]:
# Run a quick test with Case 1 up to 1000 primes
print("🧪 Running quick test: Case 1 up to 1000 primes")
print("This should take just a few seconds...\n")

with Timer("Quick test computation"):
    test_results = experiment.run_case(
        case_id=1, 
        max_prime=1000, 
        num_processes=2,
        save_results=True
    )

print(f"\n✅ Computed Frobenius elements for {len(test_results)} primes")

# Show a sample of results
print("\n📋 Sample results (first 10 primes):")
sample_primes = sorted(test_results.keys())[:10]
for p in sample_primes:
    frobenius = test_results[p]
    print(f"  p = {p:3d} → Frobenius element = {frobenius}")

## 📊 Bias Analysis: The 5-Graph Visualization

Now let's analyze the bias using our comprehensive 5-graph system. This implements the theoretical framework from the Aoki-Koyama paper.

In [ ]:
# Run a slightly larger computation for meaningful bias analysis
print("📈 Running bias analysis computation: Case 1 up to 5000 primes")
print("This will take about 10-30 seconds...\n")

# Compute Frobenius elements
bias_test_results = experiment.run_case(
    case_id=1,
    max_prime=5000,
    num_processes=4,
    save_results=True
)

print(f"✅ Computed {len(bias_test_results)} Frobenius elements")

In [ ]:
# Perform bias analysis with 5-graph visualization
print("🎨 Generating bias analysis with 5-graph visualization...")

analyzer = BiasAnalyzer(verbose=True)

# This will create the comprehensive 5-graph analysis
with Timer("Bias analysis"):
    visualization_path = analyzer.analyze_case_complete(
        case_id=1,
        data_dir="data/results"
    )

print(f"\n🎯 Bias analysis completed!")
print(f"📁 Visualization saved to: {visualization_path}")
print(f"\n📊 The analysis includes:")
print(f"  • S1: π₁/₂(x) - 8π₁/₂(x;1)   [Identity element bias]")
print(f"  • S2: π₁/₂(x) - 8π₁/₂(x;-1)  [-1 element bias]")
print(f"  • S3: π₁/₂(x) - 4π₁/₂(x;i)   [i-type element bias]")
print(f"  • S4: π₁/₂(x) - 4π₁/₂(x;j)   [j-type element bias]")
print(f"  • S5: π₁/₂(x) - 4π₁/₂(x;k)   [k-type element bias]")
print(f"\n🔴 Red lines show theoretical predictions: (M(σ) + m(σ)) log(log(x))")
print(f"⚫ Black points show actual computed values")

## 🔍 Understanding the Results

Let's interpret what the bias analysis tells us about this Q8 extension.

In [ ]:
# Load and display the analysis results
from utils import load_json

try:
    # Load the most recent analysis results
    results_file = Path("data/results") / "case_01_bias_analysis_results.json"
    if results_file.exists():
        analysis_results = load_json(results_file)
        
        print("📈 Analysis Results Summary:")
        print("============================\n")
        
        print(f"Case ID: {analysis_results['case_id']}")
        print(f"Number of primes analyzed: {analysis_results['num_primes']:,}")
        print(f"Maximum prime: {analysis_results['max_prime']:,}")
        print(f"m_ρ₀ value: {analysis_results['case_info']['m_rho0']}")
        
        print(f"\n🎯 Theoretical Bias Coefficients:")
        bias_coeffs = analysis_results['bias_coefficients']
        for key, value in bias_coeffs.items():
            print(f"  {key}: {value:+.1f}")
        
        print(f"\n📊 Regression Quality (R² values):")
        reg_stats = analysis_results.get('regression_statistics', {})
        for func in ['S1', 'S2', 'S3', 'S4', 'S5']:
            if func in reg_stats:
                r_squared = reg_stats[func].get('r_squared', 0)
                slope = reg_stats[func].get('slope', 0)
                print(f"  {func}: R² = {r_squared:.4f}, Slope = {slope:+.4f}")
        
        print(f"\n💡 Interpretation:")
        print(f"  • Higher R² values indicate better agreement with theory")
        print(f"  • Slopes should match the theoretical bias coefficients")
        print(f"  • For m_ρ₀ = {analysis_results['case_info']['m_rho0']}, we expect:")
        if analysis_results['case_info']['m_rho0'] == 0:
            print(f"    - S1 (identity): positive bias (+0.5)")
            print(f"    - S2 (-1 element): strong positive bias (+2.5)")
            print(f"    - S3, S4, S5 (i,j,k types): negative bias (-0.5)")
        else:
            print(f"    - S1 (identity): strong positive bias (+2.5)")
            print(f"    - S2 (-1 element): positive bias (+0.5)")
            print(f"    - S3, S4, S5 (i,j,k types): negative bias (-0.5)")
    else:
        print("⚠️  Analysis results not found. Run the bias analysis first.")

except Exception as e:
    print(f"⚠️  Could not load analysis results: {e}")

## 🚀 Scaling Up: Medium Scale Experiment

Now let's run a more substantial experiment with multiple cases and more primes.

In [ ]:
# Medium scale experiment: Cases 1-3 up to 50,000 primes
print("🚀 Medium Scale Experiment")
print("==========================")
print("Cases: 1, 2, 3")
print("Max prime: 50,000")
print("Expected time: 1-3 minutes")
print("")

# Ask for confirmation since this takes longer
response = input("Proceed with medium scale experiment? (y/n): ")

if response.lower() == 'y':
    print("\n🔄 Starting medium scale computation...")
    
    medium_results = experiment.run_multiple_cases(
        case_ids=[1, 2, 3],
        max_prime=50000,
        num_processes=6,
        save_results=True
    )
    
    print(f"\n✅ Medium scale computation completed!")
    for case_id, results in medium_results.items():
        print(f"  Case {case_id}: {len(results):,} Frobenius elements")
else:
    print("⏸️  Medium scale experiment skipped.")

In [ ]:
# Analyze multiple cases if we ran the medium experiment
if 'medium_results' in locals() and medium_results:
    print("📊 Analyzing multiple cases...")
    
    # Analyze each case
    case_visualizations = analyzer.analyze_multiple_cases(
        case_ids=[1, 2, 3],
        data_dir="data/results"
    )
    
    print(f"\n🎨 Generated visualizations:")
    for case_id, viz_path in case_visualizations.items():
        if viz_path:
            print(f"  Case {case_id}: {viz_path}")
    
    # Generate summary report
    print(f"\n📋 Generating summary report...")
    summary_path = analyzer.generate_summary_report([1, 2, 3])
    print(f"📁 Summary report saved to: {summary_path}")
else:
    print("⏸️  Multi-case analysis skipped (no medium results available).")

## 🎯 Complete Experiment Workflow

Here's how to run a complete experiment that includes computation, analysis, and reporting.

In [ ]:
# Complete workflow example
print("🎯 Complete Experiment Workflow Demo")
print("====================================")
print("This demonstrates the full pipeline: computation → analysis → reporting")
print("")

# Run complete experiment with small scale for demo
complete_results = experiment.run_complete_experiment(
    case_ids=[1, 2],
    max_prime=10000,
    num_processes=4,
    analyze_bias=True,
    generate_report=True
)

print(f"\n📋 Complete Experiment Results:")
print(f"================================")

if 'computation' in complete_results:
    total_primes = sum(len(case_data) for case_data in complete_results['computation'].values())
    print(f"✅ Computation: {total_primes:,} total Frobenius elements computed")

if 'analysis' in complete_results:
    successful_analyses = sum(1 for path in complete_results['analysis'].values() if path)
    print(f"📊 Analysis: {successful_analyses} bias visualizations generated")

if 'report_path' in complete_results:
    print(f"📁 Report: {complete_results['report_path']}")

duration = complete_results.get('experiment_duration', 0)
print(f"⏱️  Total time: {format_duration(duration)}")

## 🔧 Advanced Usage: Custom Analysis

For advanced users, here's how to customize the analysis and access raw data.

In [ ]:
# Access raw computation results for custom analysis
print("🔧 Advanced: Custom Analysis Example")
print("====================================")

# Get the latest results for Case 1
from fast_frobenius_calculator import load_frobenius_results
from glob import glob

# Find the most recent Case 1 data file
case1_files = glob("data/results/case_01_frobenius_*.json")
if case1_files:
    latest_file = max(case1_files, key=os.path.getmtime)
    case_id, frobenius_data, metadata = load_frobenius_results(latest_file)
    
    print(f"📁 Loaded data from: {latest_file}")
    print(f"📊 Data summary:")
    print(f"  • Case ID: {case_id}")
    print(f"  • Number of primes: {len(frobenius_data):,}")
    print(f"  • Max prime: {max(frobenius_data.keys()):,}")
    print(f"  • m_ρ₀: {metadata['computation_info']['m_rho0']}")
    
    # Analyze Frobenius element distribution
    from collections import Counter
    frobenius_counts = Counter(frobenius_data.values())
    
    print(f"\n🎲 Frobenius Element Distribution:")
    total_primes = len(frobenius_data)
    for element in sorted(frobenius_counts.keys()):
        count = frobenius_counts[element]
        percentage = 100 * count / total_primes
        print(f"  Element {element}: {count:5,} primes ({percentage:5.1f}%)")
    
    # Create a simple custom plot
    plt.figure(figsize=(10, 6))
    elements = sorted(frobenius_counts.keys())
    counts = [frobenius_counts[elem] for elem in elements]
    
    plt.bar(elements, counts, alpha=0.7, color='steelblue')
    plt.xlabel('Frobenius Element')
    plt.ylabel('Number of Primes')
    plt.title(f'Case {case_id}: Distribution of Frobenius Elements')
    plt.grid(True, alpha=0.3)
    
    # Add percentage labels
    for elem, count in zip(elements, counts):
        percentage = 100 * count / total_primes
        plt.text(elem, count + max(counts)*0.01, f'{percentage:.1f}%', 
                ha='center', va='bottom', fontsize=10)
    
    plt.tight_layout()
    plt.show()
    
else:
    print("⚠️  No Case 1 data files found. Run a computation first.")

## 🌟 Production Scale: Preparing for 10^9 Computations

For serious research, you'll want to scale up to 10^9 primes. Here's how to prepare and estimate resources.

In [ ]:
# Estimate resources for large-scale computation
from utils import estimate_memory_usage, estimate_computation_time, get_optimal_process_count

print("🌟 Production Scale Analysis")
print("============================")
print("Resource estimates for large-scale Q8 bias experiments:\n")

scales = {
    '10^5 (100K)': 100000,
    '10^6 (1M)': 1000000, 
    '10^7 (10M)': 10000000,
    '10^8 (100M)': 100000000,
    '10^9 (1B)': 1000000000
}

optimal_processes = get_optimal_process_count()

print(f"System specs:")
print(f"  • Optimal processes: {optimal_processes}")
print(f"  • Single case computations\n")

for scale_name, max_prime in scales.items():
    memory_est = estimate_memory_usage(max_prime, 1)
    time_est = estimate_computation_time(max_prime, 1, optimal_processes)
    
    print(f"📊 {scale_name} primes:")
    print(f"  • Estimated primes: {format_number(time_est['estimated_primes'])}")
    print(f"  • Memory usage: ~{memory_est['total_gb']:.1f} GB")
    print(f"  • Sequential time: {format_duration(time_est['base_time_seconds'])}")
    print(f"  • Parallel time ({optimal_processes} cores): {format_duration(time_est['parallel_time_seconds'])}")
    print(f"  • Speedup factor: {time_est['speedup_factor']:.1f}x")
    print()

print(f"💡 Recommendations:")
print(f"  • For 10^6 primes: Comfortable on most modern systems")
print(f"  • For 10^7 primes: Requires 8+ GB RAM, 8+ cores recommended")
print(f"  • For 10^8 primes: High-performance system, 16+ GB RAM")
print(f"  • For 10^9 primes: Server-class system, 32+ GB RAM, 24+ hours")
print(f"\n🔧 For production runs, consider:")
print(f"  • Running overnight or over weekends")
print(f"  • Using checkpoint/resume functionality")
print(f"  • Monitoring memory usage during computation")
print(f"  • Saving intermediate results every 10^6 primes")

In [ ]:
# Example of how to set up a large-scale computation (don't actually run this!)
print("🚀 Large-Scale Computation Setup Example")
print("========================================")
print("Here's how you would set up a 10^7 prime computation:")
print("(This is just an example - don't run it in this demo!)\n")

example_code = '''
# Large-scale experiment setup
large_experiment = QuaternionBiasExperiment(verbose=True)

# Validate setup first
validation = large_experiment.validate_experiment_setup(
    case_ids=[1, 2, 3],
    max_prime=10**7,
    num_processes=16
)

print(f"Estimated time: {validation['time_estimate_hours']:.1f} hours")
print(f"Memory needed: {validation['memory_estimate_gb']:.1f} GB")

# If validation looks good, run the computation
if input("Proceed? (y/n): ") == 'y':
    results = large_experiment.run_complete_experiment(
        case_ids=[1, 2, 3],
        max_prime=10**7,
        num_processes=16,
        analyze_bias=True,
        generate_report=True
    )
'''

print(example_code)
print("\n💾 For actual large-scale runs:")
print("1. Test with smaller scales first (10^5, 10^6)")
print("2. Monitor system resources")
print("3. Use the configuration file to adjust parameters")
print("4. Consider running cases individually for very large scales")
print("5. Save intermediate results frequently")

## 📚 Understanding the Theory

Let's dive deeper into the mathematical background of what we're computing.

In [ ]:
print("📚 Mathematical Background")
print("=========================\n")

print("🔢 Q8 Group Structure:")
print("The quaternion group Q8 = {±1, ±i, ±j, ±k} has 8 elements with:")
print("  • Identity: 1")
print("  • Inverse: -1")
print("  • Quaternion units: ±i, ±j, ±k")
print("  • Relations: i² = j² = k² = ijk = -1")
print()

print("🎯 Conjugacy Classes:")
case1 = get_case(1)
conjugacy_sizes = case1.get_conjugacy_sizes()
print("Q8 has 5 conjugacy classes:")
conjugacy_description = {
    'g0': '{1} - Identity',
    'g1': '{-1} - Central element', 
    'g2': '{i, -i} - i-type',
    'g3': '{j, -j} - j-type',
    'g4': '{k, -k} - k-type'
}

for g, size in conjugacy_sizes.items():
    if g in conjugacy_description:
        print(f"  • {conjugacy_description[g]} (size {size})")
print()

print("📊 Bias Functions:")
print("We study 5 bias functions S1-S5 based on weighted prime counting:")
print("  • π₁/₂(x) = Σ(p≤x) 1/√p  [total weighted count]")
print("  • π₁/₂(x;σ) = Σ(p≤x, Frob(p)=σ) 1/√p  [count for specific Frobenius]")
print()
print("The bias functions are:")
print("  • S1 = π₁/₂(x) - 8π₁/₂(x;1)   [identity bias, factor 8 = |Q8|/|C₁|]")
print("  • S2 = π₁/₂(x) - 8π₁/₂(x;-1)  [-1 bias, factor 8 = |Q8|/|C₋₁|]")
print("  • S3 = π₁/₂(x) - 4π₁/₂(x;i)   [i-type bias, factor 4 = |Q8|/|Cᵢ|]")
print("  • S4 = π₁/₂(x) - 4π₁/₂(x;j)   [j-type bias, factor 4 = |Q8|/|Cⱼ|]")
print("  • S5 = π₁/₂(x) - 4π₁/₂(x;k)   [k-type bias, factor 4 = |Q8|/|Cₖ|]")
print()

print("🎲 Theoretical Prediction (Aoki-Koyama):")
print("Under the Deep Riemann Hypothesis:")
print("  Sᵢ(x) ≈ (M(σ) + m(σ)) log(log(x))")
print()
print("For Q8 extensions, the bias coefficients depend on m_ρ₀:")

# Show bias coefficients for both m_ρ₀ values
case1_coeffs = get_case(1).get_bias_coefficients()  # m_ρ₀ = 0
case2_coeffs = get_case(2).get_bias_coefficients()  # m_ρ₀ = 1

print("\n  m_ρ₀ = 0 cases:")
for coeff, value in case1_coeffs.items():
    if coeff.startswith('g') and int(coeff[1]) <= 4:
        func_name = ['S1', 'S2', 'S3', 'S4', 'S5'][int(coeff[1])]
        print(f"    {func_name}: {value:+.1f}")

print("\n  m_ρ₀ = 1 cases:")
for coeff, value in case2_coeffs.items():
    if coeff.startswith('g') and int(coeff[1]) <= 4:
        func_name = ['S1', 'S2', 'S3', 'S4', 'S5'][int(coeff[1])]
        print(f"    {func_name}: {value:+.1f}")

print("\n💡 Interpretation:")
print("  • Positive coefficients → bias toward that element type")
print("  • Negative coefficients → bias against that element type")
print("  • The magnitude indicates the strength of the bias")
print("  • Good agreement between slopes and theory validates the conjecture")

## 🎓 Next Steps and Further Exploration

You've completed the demo! Here are suggestions for further exploration.

In [ ]:
print("🎓 Congratulations! You've completed the Q8 Prime Bias Demo")
print("=========================================================\n")

print("🔍 What you've learned:")
print("  ✅ How to run Frobenius element computations")
print("  ✅ How to analyze bias with 5-graph visualizations")
print("  ✅ How to interpret results and compare with theory")
print("  ✅ How to scale up for large-scale research")
print("  ✅ Understanding of Q8 group structure and bias theory")
print()

print("🚀 Next Steps for Research:")
print()
print("1. 📊 Scale up computations:")
print("   • Try 10^5 primes for better bias visualization")
print("   • Run all 13 cases for comprehensive comparison")
print("   • Use parallel processing for 10^6+ prime computations")
print()
print("2. 🔬 Deep analysis:")
print("   • Compare cases with different m_ρ₀ values")
print("   • Study convergence rates and R² statistics")
print("   • Investigate ramified prime effects")
print()
print("3. 📈 Advanced visualization:")
print("   • Create interactive plots with Plotly")
print("   • Generate multi-case comparison reports")
print("   • Export data for external analysis")
print()
print("4. 🧪 Experimental extensions:")
print("   • Test other polynomial families")
print("   • Compare with SageMath implementations")
print("   • Implement additional statistical tests")
print()

print("📚 Useful commands for your research:")
print()
print("# Run a single case to 10^6 primes:")
print("experiment.run_case(case_id=1, max_prime=10**6, num_processes=8)")
print()
print("# Run all 13 cases (small scale):")
print("experiment.run_multiple_cases(list(range(1,14)), max_prime=50000)")
print()
print("# Complete workflow with analysis:")
print("experiment.run_complete_experiment([1,2,3], max_prime=100000)")
print()
print("# Custom analysis:")
print("analyzer.analyze_multiple_cases([1,2,3,4,5])")
print("analyzer.generate_summary_report(list(range(1,14)))")
print()

print("📁 Generated files to explore:")
print("  • data/results/ - Computation results (JSON)")
print("  • graphs/output/ - Bias analysis visualizations (PDF)")
print("  • logs/ - Computation logs and progress")
print()

print("🤝 Contributing to research:")
print("  • Share interesting findings with the mathematics community")
print("  • Report any computational anomalies or unexpected patterns")
print("  • Contribute optimizations for even larger scale computations")
print("  • Extend to other Galois groups beyond Q8")
print()

print("📞 Questions or issues?")
print("  • Check the README.md for detailed documentation")
print("  • Review the paper: Aoki & Koyama (2022) arXiv:2203.12266")
print("  • Create issues on the GitHub repository")
print()

print("🌟 Happy researching! The world of Q8 prime bias awaits your exploration!")

---

## 📖 References

- **Main Paper**: Aoki, M. and Koyama, S. (2022). *Chebyshev's Bias against Splitting and Principal Primes in Global Fields*. arXiv:2203.12266

- **Implementation**: This notebook demonstrates the complete numerical experiment environment for studying Chebyshev bias in Q8 Galois extensions, with optimizations for computations up to 10^9 primes.

- **Theory**: The bias functions S1-S5 implement the weighted prime counting approach from the Aoki-Koyama theoretical framework, providing numerical verification of bias predictions under the Deep Riemann Hypothesis.

---